In [1]:
import math
import numpy as np

In [2]:
def prefix_sum(a):
    b = np.zeros(len(a) + 1, dtype=np.int32)
    for i in range(len(a)):
        b[i+1] = b[i] + a[i]
    return b

In [3]:
a = np.array([3, 5, 3, 1, 6])
b = np.random.randint(1, 10, 2**16, dtype=np.int32)
c = np.random.randint(1, 10, 2**22, dtype=np.int32)

In [4]:
%timeit -r2 -n1 prefix_sum(c)

1.7 s ± 10.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [5]:
#### Parallel prefix sum in python WITHOUT THREADS ####

In [8]:
def summator(array, level, elements):
    for sum_index in elements:
        array[sum_index] = array[sum_index] + array[sum_index - pow(2, (level-1))]
    return array


def down_summator(array, level, elements):
    for sum_index in elements:
        val = array[sum_index]
        array[sum_index] = array[sum_index] + array[sum_index - pow(2, (level-1))]
        array[sum_index - pow(2, (level-1))] = val
    return array


def process(array, func, start, end, inc, max_cores):
    length = len(array)
    for level in range(start, end, inc):
        processing_field = pow(2, level)
        core_number = int(length / processing_field)
        if core_number > max_cores:
            core_number = max_cores
            processing_field = int(length / core_number)
        #print("Level:", level, " Used cores:", core_number)
        for core in range(core_number):
            lbound = processing_field * core
            rbound = processing_field * (core + 1)
            elements = range(lbound + pow(2, level) - 1, rbound, pow(2, level))
            #print("Level:", level, "   Core: ", core, "   Processing_field: ", processing_field, "   L: ", lbound, "   R: ", rbound, elements)
            func(array, level, elements)
    return array


def shift_bit_length(x):
    return 1 << (x-1).bit_length()


def pad(array):
    pad_left = shift_bit_length(len(array)) - len(array)
    return np.pad(array, (pad_left, 0), mode='constant')


def parallel_prefix_sum(array, max_cores=8):
    length = len(array)
    depth = math.ceil(math.log(length, 2))
    #print("Depth:", depth)

    b = np.copy(array)
    b = process(b, summator, start=1, end=depth + 1, inc=1, max_cores=max_cores)
    b = np.insert(b, length - 1, 0)
    #print("After summator:", b)
    b = process(b, down_summator, start=depth, end=0, inc=-1, max_cores=max_cores)
    #print("After down summator:", b)
    return b

In [9]:
%timeit -r2 -n1 parallel_prefix_sum(c)

8.66 s ± 90.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [13]:
### Just simple prefix sum works much faster because it don't need all calculation overhead
### Here I don't used multiprocessing because of GIL it will work even more slowly

### Let's check that everything was calculated successfully:

In [14]:
np.sum(prefix_sum(b))

10753681620

In [15]:
np.sum(parallel_prefix_sum(b))

10753681620

In [17]:
prefix_sum(b)[0:12]

array([ 0,  4,  8, 14, 22, 31, 37, 39, 40, 49, 55, 58], dtype=int32)

In [19]:
parallel_prefix_sum(b)[0:12]

array([ 0,  4,  8, 14, 22, 31, 37, 39, 40, 49, 55, 58], dtype=int32)